In [2]:
import os
from pathlib import Path
from PIL import Image
from typing import Any, Literal

from prettytable import PrettyTable
import skimage as ski
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchmetrics as tm
from torchvision.models import get_model, get_weight
from torchvision.datasets import DatasetFolder
from torchvision.transforms import v2

# Setup

In [3]:
DATASET_PATHS = {
    'fog-detection': Path('./datasets/fog-detection-dataset-prepared'),
    'fog-or-smog': Path('./datasets/fog-or-smog-detection-dataset-prepared'),
    'foggy-cityscapes': Path('./datasets/foggy-cityscapes-image-dataset-prepared')
}

In [9]:
get_weight("RegNet_X_32GF_Weights.DEFAULT").transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [11]:
def load_image(path: str):
    img = ski.io.imread(path)
    if img.ndim == 2:  # Handle grayscale
        img = ski.color.gray2rgb(img)
    if img.shape[-1] == 4:  # Handle RGBA
        img = ski.color.rgba2rgb(img)
    img = ski.util.img_as_ubyte(img)
    img = img.squeeze()
    
    return Image.fromarray(img)

train_transforms = v2.Compose([

    v2.ToImage(),
    v2.Grayscale(3),
    v2.RandomResizedCrop(size=(224,224), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])
])

test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Grayscale(3),
    v2.Resize(size=(224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.25, 0.25, 0.25])
])

def get_dataloader(
    path: Path | str,
    transform: Any | None = None,
    batch_size: int = 32,
    shuffle: bool = False,
    num_workers: int = 0,
    pin_memory: bool = True,
):
    return DataLoader(
        dataset=DatasetFolder(
            root=path,
            loader=load_image,
            extensions=[".jpg", ".png", ".jpeg"],
            transform=transform,
        ),
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        persistent_workers=True if num_workers > 0 else False,
    )

In [12]:
def recursive_getattr(obj: object, name: str):
    names = name.split(".")
    for name in names:
        obj = getattr(obj, name)
    return obj

def recursive_setattr(obj: object, value: Any, name: str):
    names = name.split(".")
    for name in names[:-1]:
        obj = getattr(obj, name)
    setattr(obj, names[-1], value)

In [13]:
class CNNClassifier(L.LightningModule):
    def __init__(
        self,
        model_name: str,
        num_classes: int,
        loss: nn.Module | None = None,
        learning_rate: float= 1e-4,
        optimizer_name: Literal['adam', 'adamw'] = 'adam',
        weights: str | bool = False,
    ):
        super().__init__()
        self.save_hyperparameters()

        self.learning_rate = learning_rate
        self.optimizer_name = optimizer_name

        self.model_name = model_name
        self.num_classes = num_classes
        self.weights = weights
        if weights == True:  # noqa: E712
            self.weights = f"{self.model_name}_Weights.DEFAULT"
        self.model = self._get_model()

        if loss is not None:
            self.criterion= loss
        else:
            self.criterion = nn.CrossEntropyLoss()

        task = "multiclass" if num_classes > 2 else "binary"
        
        self.train_metrics = tm.MetricCollection({
            "accuracy": tm.classification.Accuracy(task=task, num_classes=num_classes),
            "f1": tm.classification.F1Score(task=task, num_classes=num_classes),
            "precision": tm.classification.Precision(task=task, num_classes=num_classes),
            "recall": tm.classification.Recall(task=task, num_classes=num_classes),
        }, prefix="train_")
        self.validation_metrics = self.train_metrics.clone(prefix="val_")
        self.test_metrics = self.train_metrics.clone(prefix="test_")

    def _get_model(self):
        model = get_model(self.model_name, weights=self.weights)

        classifier_path = None
        classifier_in_features = None
        if hasattr(model, 'fc') and isinstance(model.fc, nn.Linear):
            classifier_path = "fc"
            classifier_in_features = model.fc.in_features
        elif hasattr(model, 'classifier'):
            if isinstance(model.classifier, nn.Linear):
                classifier_path = 'classifier'
                classifier_in_features = model.classifier.in_features
            elif isinstance(model.classifier, nn.Sequential):
                for name, module in reversed(list(model.classifier.named_children())):
                    if isinstance(module, nn.Linear):
                        classifier_path = f"classifier.{name}"
                        classifier_in_features = module.in_features
                        break
        elif hasattr(model, 'heads') and hasattr(model.heads, 'head') and isinstance(model.heads.head, nn.Linear):
            classifier_path = 'heads.head'
            classifier_in_features = model.heads.head.in_features

        
        recursive_setattr(
            model, nn.Linear(classifier_in_features, self.num_classes), classifier_path
        )

        return model
    
    def forward(self, x):
        return self.model(x)
    
    def _common_step(self, batch, batch_idx):
        images, labels = batch
        logits = self(images)
        loss = self.criterion(logits, labels)
        preds = torch.argmax(logits, dim=1)
        return loss, preds, labels
    
    def training_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)

        self.log("train_loss", loss)
        self.log_dict(self.train_metrics(preds, labels))

        return loss
    
    def on_train_epoch_end(self):
        self.train_metrics.reset()

    def validation_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.validation_metrics.update(preds, labels)
        self.log("val_loss", loss)
        return loss
    
    def on_validation_epoch_end(self):
        self.log_dict(self.validation_metrics.compute())
        self.validation_metrics.reset()
    

    def test_step(self, batch, batch_idx, dataloader_idx=0):
        loss, preds, labels = self._common_step(batch, batch_idx)
        self.test_metrics.update(preds, labels)
        return loss
    
    def on_test_epoch_end(self):
        self.log_dict(self.test_metrics.compute())
        self.test_metrics.reset()

    def configure_optimizers(self):
        if self.optimizer_name == 'adam':
            optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_name == 'adamw':
            optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}.")
        return optimizer

# Training

## ResNet18

In [15]:
SAVE_DIR = Path('runs/classify/CNN-GRAYSCALE')
MODEL_NAME = "ResNet18"
DATASET = 'fog-detection'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=False,
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN-GRAYSCALE/ResNet18-fog-detection/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | t

Epoch 8: 100%|██████████| 12/12 [00:09<00:00,  1.31it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:04<00:00,  0.90it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9056603908538818     │
│          test_f1          │    0.9285714030265808     │
│      test_precision       │    0.8666666746139526     │
│        test_recall        │            1.0            │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8556034564971924     │
│          test_f1          │    0.8714011311531067     │
│      test_precision       │    0.8078292012214661     │
│        test_recall        │    0.9458333253860474     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:13<00:00,  0.51it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.5151515007019043     │
│          test_f1          │    0.6470588445663452     │
│      test_precision       │    0.5086705088615417     │
│        test_recall        │    0.8888888955116272     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.9057,0.9286,0.8667,1.0
fog-or-smog,0.8556,0.8714,0.8078,0.9458
foggy-cityscapes,0.5152,0.6471,0.5087,0.8889


In [16]:
SAVE_DIR = Path('runs/classify/CNN-GRAYSCALE')
MODEL_NAME = "ResNet18"
DATASET = 'fog-or-smog'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=False,
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN-GRAYSCALE/ResNet18-fog-or-smog/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      | tra

Epoch 8: 100%|██████████| 52/52 [00:09<00:00,  5.48it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:04<00:00,  0.80it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.7547169923782349     │
│          test_f1          │           0.75            │
│      test_precision       │            1.0            │
│        test_recall        │    0.6000000238418579     │
└───────────────────────────┴───────────────────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.9353448152542114     │
│          test_f1          │    0.9342105388641357     │
│      test_precision       │    0.9861111044883728     │
│        test_recall        │     0.887499988079071     │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:15<00:00,  0.45it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6868686676025391     │
│          test_f1          │    0.6395348906517029     │
│      test_precision       │    0.7534246444702148     │
│        test_recall        │    0.5555555820465088     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.7547,0.75,1.0,0.6
fog-or-smog,0.9353,0.9342,0.9861,0.8875
foggy-cityscapes,0.6869,0.6395,0.7534,0.5556


In [ ]:
SAVE_DIR = Path('runs/classify/CNN-GRAYSCALE')
MODEL_NAME = "ResNet18"
DATASET = 'foggy-cityscapes'
VERSION = 1

trainer = L.Trainer(
    max_epochs=10,
    logger=L.pytorch.loggers.TensorBoardLogger(
        save_dir=SAVE_DIR,
        name=f"{MODEL_NAME}-{DATASET}",
        version=VERSION,
    ),
    callbacks=[
        L.pytorch.callbacks.early_stopping.EarlyStopping(
            monitor="val_loss", mode="min",
            patience=5,
            verbose=False    
        ),
        L.pytorch.callbacks.ModelCheckpoint(
            monitor="val_f1", mode="max",
            dirpath=SAVE_DIR / f"{MODEL_NAME}-{DATASET}" / f"version_{VERSION}",
            filename="{epoch}-{val_loss:.2f}-{val_f1:.2f}"
        )
    ],
    log_every_n_steps=1
)

model = CNNClassifier(
    model_name=MODEL_NAME,
    num_classes=2,
    weights=False,
)

trainer.fit(
    model,
    train_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'train',
        transform=train_transforms,
        batch_size=32,
        shuffle=True,
        num_workers=11
    ),
    val_dataloaders=get_dataloader(
        path=DATASET_PATHS[DATASET] / 'val',
        transform=test_transforms,
        batch_size=32,
        shuffle=False,
        num_workers=11
    ),
)

model = CNNClassifier.load_from_checkpoint(
    trainer.checkpoint_callbacks[0].best_model_path
)

res = {
    dataset_name: trainer.test(model, get_dataloader(path=path / 'test', transform=test_transforms))[0]
    for dataset_name, path in DATASET_PATHS.items()
}

table = PrettyTable()
table.field_names = [
    "Dataset", *list(next(iter(res.values())).keys())
]
table.add_rows([
    [dataset, *[round(m, 4) for m in metrics.values()]] for dataset, metrics in res.items()
])
table

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/next/magisterka/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/next/magisterka/runs/classify/CNN-GRAYSCALE/ResNet18-foggy-cityscapes/version_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params | Mode 
----------------------------------------------------------------
0 | model              | ResNet           | 11.2 M | train
1 | criterion          | CrossEntropyLoss | 0      | train
2 | train_metrics      | MetricCollection | 0      

Epoch 8: 100%|██████████| 22/22 [00:18<00:00,  1.21it/s, v_num=1]          


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:07<00:00,  0.52it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.698113203048706     │
│          test_f1          │    0.7611940503120422     │
│      test_precision       │     0.739130437374115     │
│        test_recall        │    0.7846153974533081     │
└───────────────────────────┴───────────────────────────┘

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/next/magisterka/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 7/7 [00:14<00:00,  0.47it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8383838534355164     │
│          test_f1          │    0.8399999737739563     │
│      test_precision       │    0.8316831588745117     │
│        test_recall        │    0.8484848737716675     │
└───────────────────────────┴───────────────────────────┘

Dataset,test_accuracy,test_f1,test_precision,test_recall
fog-detection,0.6981,0.7612,0.7391,0.7846
fog-or-smog,0.7586,0.8049,0.6916,0.9625
foggy-cityscapes,0.8384,0.84,0.8317,0.8485


: 